In [115]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

In [116]:
data = pd.read_csv('SPC_1500_40.csv', header=0)

Only keep the necessary columns

In [117]:
data = data[['EFFECTIVE DATE', 'COMPANY', 'CUSIP', 'SALES-PRICE', 'BV-PRICE', 'EPS_MINUS_3M-PRICE', 'ROA', \
             'EPS-PRICE', 'PRICE MOM', 'NEXT MONTH TR']]

In [118]:
data.head()

,EFFECTIVE DATE,COMPANY,CUSIP,SALES-PRICE,BV-PRICE,EPS_MINUS_3M-PRICE,ROA,EPS-PRICE,PRICE MOM,NEXT MONTH TR
0,19941230,A.G. Edwards,281760108,71.035611,0.484281,0.077778,7.124138,0.081667,-0.060000,0.020833373
1,19941230,AFLAC Inc.,001055102,191.641594,0.045786,0.007188,1.819224,0.007188,-0.003956,0.08203125
2,19941230,AMRESCO Inc.,031909104,23.286519,3.566218,0.640000,19.328806,0.666667,-0.188811,-0.074074087
3,19941230,Aetna Inc. (Old),008140105,380.604775,0.259135,0.025889,0.507874,0.023979,-0.029810,0.065039754
4,19941230,Ahmanson (H.F.) & Co.,008677106,NaN,1.569836,0.120930,0.466019,0.112868,-0.119216,0.015503883


Check data types

In [119]:
data.dtypes

EFFECTIVE DATE          int64
COMPANY                object
CUSIP                  object
SALES-PRICE           float64
BV-PRICE              float64
EPS_MINUS_3M-PRICE    float64
ROA                   float64
EPS-PRICE             float64
PRICE MOM             float64
NEXT MONTH TR          object
dtype: object

NEXT MONTH TR needs to be coerced to float

In [120]:
data['NEXT MONTH TR'] = pd.to_numeric(data['NEXT MONTH TR'], errors='coerce')

In [121]:
data['NEXT MONTH TR'].dtypes

dtype('float64')

Check dimensions

In [122]:
data.shape

(54435, 10)

Verify that the dates are all end-of-month

In [123]:
list(set(data['EFFECTIVE DATE'].astype(str).str[-2:]))

['31', '28', '26', '29', '27', '30']

In [124]:
print('The data range from', data['EFFECTIVE DATE'].min(), 'to', data['EFFECTIVE DATE'].max())

The data range from 19941230 to 20150731


Check to see how many nulls there are

In [125]:
data.iloc[:, 3:].isna().sum()

SALES-PRICE            562
BV-PRICE              1327
EPS_MINUS_3M-PRICE    1905
ROA                   1628
EPS-PRICE             1827
PRICE MOM               25
NEXT MONTH TR          424
dtype: int64

Check to see how many rows are free of nulls

In [126]:
nonnull_rows = sum(data.iloc[:, 3:].isna().sum(axis=1) == 0)
print(nonnull_rows)

50233


In [127]:
print('Approximately {0:.0f} percent of rows don\'t contain any null datapoints.  This is acceptable to the extent that any \
rows containing at least one null entry can be dropped'.format(100*round(nonnull_rows / data.shape[0],2)))

Approximately 92 percent of rows don't contain any null datapoints.  This is acceptable to the extent that any rows containing at least one null entry can be dropped


Drop rows containing any nulls

In [128]:
data.dropna(inplace=True)

data.reset_index(inplace=True)

In [129]:
data.drop('index', axis=1, inplace=True)

In [130]:
data.head()

,EFFECTIVE DATE,COMPANY,CUSIP,SALES-PRICE,BV-PRICE,EPS_MINUS_3M-PRICE,ROA,EPS-PRICE,PRICE MOM,NEXT MONTH TR
0,19941230,A.G. Edwards,281760108,71.035611,0.484281,0.077778,7.124138,0.081667,-0.060000,0.020833
1,19941230,AFLAC Inc.,001055102,191.641594,0.045786,0.007188,1.819224,0.007188,-0.003956,0.082031
2,19941230,AMRESCO Inc.,031909104,23.286519,3.566218,0.640000,19.328806,0.666667,-0.188811,-0.074074
3,19941230,Aetna Inc. (Old),008140105,380.604775,0.259135,0.025889,0.507874,0.023979,-0.029810,0.065040
4,19941230,Alex Brown Inc,013902101,19.933761,0.573544,0.098765,5.829485,0.093498,0.027149,0.149794


In [131]:
print('The number of constituents at each month ranges from', \
      data.groupby(['EFFECTIVE DATE'])['EFFECTIVE DATE'].count().min(), 'to', \
      data.groupby(['EFFECTIVE DATE'])['EFFECTIVE DATE'].count().max())

The number of constituents at each month ranges from 137 to 292


Engineer new feature: EPS-MOMENTUM =  EPS-PRICE / EPS_MINUS_3M-PRICE - 1

In [132]:
data['EPS-MOMENTUM'] = data['EPS-PRICE'] / data['EPS_MINUS_3M-PRICE'] - 1

Check if NaN in the new feature (on account of, e.g., dividing by zero)

In [133]:
data['EPS-MOMENTUM'].isnull().any()

True

Coerce NaN's in the new feature with 0

In [134]:
data['EPS-MOMENTUM'].fillna(0, inplace=True)

Check if infinity entries in the new feature

In [135]:
sum(data['EPS-MOMENTUM'] == np.inf) + sum(data['EPS-MOMENTUM'] == -np.inf)

41

Coerce infinity to 9999, in order to conduct EDA and statistical analyses

In [136]:
data['EPS-MOMENTUM'][(data['EPS-MOMENTUM'] == np.inf) | (data['EPS-MOMENTUM'] == -np.inf)] = 9999

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Rearrange/swap the last two columns so that the target (NEXT MONTH TR) is rightmost

In [137]:
cols = data.columns.tolist()

feature_col = cols[-1]
target_col = cols[-2]

cols = cols[:-2]

cols.append(feature_col)
cols.append(target_col)

data = data[cols]

Delete EPS_MINUS_3M-PRICE, as it was only needed in order to engineer EPS-MOMENTUM

In [138]:
data.drop('EPS_MINUS_3M-PRICE', axis=1, inplace=True)

In [139]:
data.head()

,EFFECTIVE DATE,COMPANY,CUSIP,SALES-PRICE,BV-PRICE,ROA,EPS-PRICE,PRICE MOM,EPS-MOMENTUM,NEXT MONTH TR
0,19941230,A.G. Edwards,281760108,71.035611,0.484281,7.124138,0.081667,-0.060000,0.050000,0.020833
1,19941230,AFLAC Inc.,001055102,191.641594,0.045786,1.819224,0.007188,-0.003956,0.000000,0.082031
2,19941230,AMRESCO Inc.,031909104,23.286519,3.566218,19.328806,0.666667,-0.188811,0.041667,-0.074074
3,19941230,Aetna Inc. (Old),008140105,380.604775,0.259135,0.507874,0.023979,-0.029810,-0.073770,0.065040
4,19941230,Alex Brown Inc,013902101,19.933761,0.573544,5.829485,0.093498,0.027149,-0.053333,0.149794


Create function that groups by date and discretizes the (continuous) features into 5 quantiles

In [140]:
def discretize(df, data_col, date_col = 'EFFECTIVE DATE'):
    pct = df.groupby([date_col])[data_col].transform(lambda x: rankdata(x, 'average')/len(x))
    df.new_col_name = [1 if x < 0.2 else \
                                2 if x < 0.4 else \
                                3 if x < 0.6 else \
                                4 if x < 0.8 else \
                                5 for x in pct]
    return df.new_col_name

In [141]:
data['SALES-PRICE DISCRETE'] = discretize(data, 'SALES-PRICE')
data['BV-PRICE DISCRETE'] = discretize(data, 'BV-PRICE')
data['ROA DISCRETE'] = discretize(data, 'ROA')
data['EPS-PRICE DISCRETE'] = discretize(data, 'EPS-PRICE')
data['PRICE MOM DISCRETE'] = discretize(data, 'PRICE MOM')
data['EPS-MOMENTUM DISCRETE'] = discretize(data, 'EPS-MOMENTUM')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys


In [142]:
data.head()

,EFFECTIVE DATE,COMPANY,CUSIP,SALES-PRICE,BV-PRICE,ROA,EPS-PRICE,PRICE MOM,EPS-MOMENTUM,NEXT MONTH TR,SALES-PRICE DISCRETE,BV-PRICE DISCRETE,ROA DISCRETE,EPS-PRICE DISCRETE,PRICE MOM DISCRETE,EPS-MOMENTUM DISCRETE
0,19941230,A.G. Edwards,281760108,71.035611,0.484281,7.124138,0.081667,-0.060000,0.050000,0.020833,4,3,5,4,3,5
1,19941230,AFLAC Inc.,001055102,191.641594,0.045786,1.819224,0.007188,-0.003956,0.000000,0.082031,5,1,4,1,5,3
2,19941230,AMRESCO Inc.,031909104,23.286519,3.566218,19.328806,0.666667,-0.188811,0.041667,-0.074074,2,5,5,5,1,5
3,19941230,Aetna Inc. (Old),008140105,380.604775,0.259135,0.507874,0.023979,-0.029810,-0.073770,0.065040,5,2,1,1,4,1
4,19941230,Alex Brown Inc,013902101,19.933761,0.573544,5.829485,0.093498,0.027149,-0.053333,0.149794,2,3,5,4,5,1


Create function that discretizes the target -- 1 if the next month's return lies above the median, 0 otherwise

In [143]:
def discretize_target(df, target_col, date_col = 'EFFECTIVE DATE'):
    pct = df.groupby([date_col])[target_col].transform(lambda x: rankdata(x, 'average')/len(x))
    df.new_col_name = [1 if x > 0.5 else \
                                0 for x in pct]
    return df.new_col_name

In [144]:
data['NEXT MONTH TR DISCRETE'] = discretize_target(data, 'NEXT MONTH TR')

In [145]:
data.head()

,EFFECTIVE DATE,COMPANY,CUSIP,SALES-PRICE,BV-PRICE,ROA,EPS-PRICE,PRICE MOM,EPS-MOMENTUM,NEXT MONTH TR,SALES-PRICE DISCRETE,BV-PRICE DISCRETE,ROA DISCRETE,EPS-PRICE DISCRETE,PRICE MOM DISCRETE,EPS-MOMENTUM DISCRETE,NEXT MONTH TR DISCRETE
0,19941230,A.G. Edwards,281760108,71.035611,0.484281,7.124138,0.081667,-0.060000,0.050000,0.020833,4,3,5,4,3,5,0
1,19941230,AFLAC Inc.,001055102,191.641594,0.045786,1.819224,0.007188,-0.003956,0.000000,0.082031,5,1,4,1,5,3,1
2,19941230,AMRESCO Inc.,031909104,23.286519,3.566218,19.328806,0.666667,-0.188811,0.041667,-0.074074,2,5,5,5,1,5,0
3,19941230,Aetna Inc. (Old),008140105,380.604775,0.259135,0.507874,0.023979,-0.029810,-0.073770,0.065040,5,2,1,1,4,1,1
4,19941230,Alex Brown Inc,013902101,19.933761,0.573544,5.829485,0.093498,0.027149,-0.053333,0.149794,2,3,5,4,5,1,1


In [146]:
data.to_csv('wrangled_data.csv', index=False)